In [1]:
import pandas as pd
import numpy as np

In [2]:
# create dataframe same as the example
df1 = pd.DataFrame({'Id':list(range(1,9)),
                   'Class':[1,1,2,1,2,3,3,1],
                   'Grade':[86,95,89,83,86,92,86,88]})

In [3]:
df1

,Id,Class,Grade
0,1,1,86
1,2,1,95
2,3,2,89
3,4,1,83
4,5,2,86
5,6,3,92
6,7,3,86
7,8,1,88


In [4]:
# use rank
pd.DataFrame(df1.groupby('Class')['Grade'].rank(ascending=False)).reset_index().merge(df1.reset_index(),how='right',on='index').sort_values(['Class','Grade_x']).rename(columns={'Grade_x':'Ranking',
'Grade_y':'Grade'}).drop('index',axis=1)

,Ranking,Id,Class,Grade
1,1.0,2,1,95
7,2.0,8,1,88
0,3.0,1,1,86
3,4.0,4,1,83
2,1.0,3,2,89
4,2.0,5,2,86
5,1.0,6,3,92
6,2.0,7,3,86


In [5]:
# use rolling methods: comparing to SQL
# equivalent SQL syntax: 
# SELECT *,
# SUM(Grade) OVER(PARTITION BY Class ORDER BY Id) AS current_sum,
pd.DataFrame(df1.sort_values('Class').groupby('Class')['Grade'].agg(np.cumsum)).reset_index().merge(df1.reset_index(),on='index').rename(columns={'Grade_x':'current_sum',
'Grade_y':'Grade'}).drop('index',axis=1)[['Id','Class','Grade','current_sum']]

,Id,Class,Grade,current_sum
0,1,1,86,86
1,2,1,95,181
2,4,1,83,264
3,8,1,88,352
4,3,2,89,89
5,5,2,86,175
6,6,3,92,92
7,7,3,86,178


In [6]:
# if there is no partition, then directly use np.cumsum(same as the example)
pd.DataFrame(df1['Grade'].agg(np.cumsum)).reset_index().merge(df1.reset_index(),on='index').rename(columns={'Grade_x':'current_sum',
'Grade_y':'Grade'}).drop('index',axis=1)[['Id','Class','Grade','current_sum']]

,Id,Class,Grade,current_sum
0,1,1,86,86
1,2,1,95,181
2,3,2,89,270
3,4,1,83,353
4,5,2,86,439
5,6,3,92,531
6,7,3,86,617
7,8,1,88,705


**Demonstrate implementing UNBOUNDED moving average**

which is equivalent to 

SELECT *,

AVG(Grade) OVER(PARTITION BY Class ORDER BY Id) AS current_sum

FROM T1

In [7]:
def cumavg(data):
    return [np.mean(data[:i]) for i in range(len(data)+1) if len(data[:i])>0]

def make_agg_func(func, name, *args, **kwargs):
           def wrapper(x):
               return func(x, *args, **kwargs)
           wrapper.__name__ = name
           return wrapper
           
make_agg_func(cumavg,'cumavg')

<function __main__.make_agg_func.<locals>.wrapper(x)>

In [10]:
# # create a df for each group and then combine together
# def create_df(group_name):
#     g_keys = df1.sort_values('Class').groupby('Class')['Grade'].get_group(group_name).keys()
#     g_values = df1.sort_values('Class').groupby('Class')['Grade'].get_group(group_name).agg(cumavg)
#     g_df = pd.DataFrame({'index':g_keys,
#                          'cumavg':g_values})
#     return g_df

In [11]:
# # inspect group1
# dfs = [create_df(i) for i in range(1,4)]
# dfs[0]

,index,cumavg
0,0,86.0
1,1,90.5
2,3,88.0
3,7,88.0


In [10]:
# pd.concat(dfs).merge(df1.reset_index(),how='inner',on='index').drop('index',axis=1)[['Id','Class','Grade','cumavg']]

,Id,Class,Grade,cumavg
0,1,1,86,86.0
1,2,1,95,90.5
2,4,1,83,88.0
3,8,1,88,88.0
4,3,2,89,89.0
5,5,2,86,87.5
6,6,3,92,92.0
7,7,3,86,89.0


In [8]:
# packaging
# create a df for each group and then combine together
def create_df(df, agg_col, partition, order, agg_fuc, group_name, ascending=True):
    g_keys = df.sort_values(order,ascending=ascending).groupby(partition)[agg_col].get_group(group_name).keys()
    g_values = df.sort_values(partition).groupby(partition)[agg_col].get_group(group_name).agg(agg_fuc)
    g_df = pd.DataFrame({'index':g_keys,
                         '%s' %agg_fuc.__name__:g_values})
    return g_df

def sql_over(df, agg_col, partition, order, agg_fuc, ascending=True):
    group_names = list(df[partition].unique())
    dfs = [create_df(df, agg_col, partition, order, agg_fuc, group_name, ascending) for group_name in group_names]
    result = pd.concat(dfs).merge(df.reset_index(),how='inner',on='index').drop('index',axis=1)[list(df.columns)+[agg_fuc.__name__]]
    return result

In [10]:
sql_over(df1,'Grade', 'Class', 'Class',cumavg,ascending=False)

,Id,Class,Grade,cumavg
0,1,1,86,86.0
1,2,1,95,90.5
2,4,1,83,88.0
3,8,1,88,88.0
4,3,2,89,89.0
5,5,2,86,87.5
6,6,3,92,92.0
7,7,3,86,89.0
